In [1]:
import yfinance as yf # $ pip install yfinance --upgrade --no-cache-dir
import pandas as pd
import numpy as np
import math
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn import datasets, linear_model
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from Back_Test import *
from Indicator import *
import pandas as pd

pd.set_option('display.max_rows', 50)

ytd = datetime.now() - timedelta(days=1)
ytd.strftime("%Y-%m-%d")

start_date = "2008-01-01"
stop_date = "2020-12-28"
cash = 100000000

# get trading date
date_df = trading_dates(start_date, stop_date)

# create context variable
context = Backtest(cash, date_df[0])
context.date_df = date_df

asset_list = "SPY GLD TLT ^VIX ^TNX ^FVX HYG"
context.load_asset_list(asset_list,"2007-07-02","2013-01-01")

[*********************100%***********************]  7 of 7 completed


In [2]:
print(context.data["Adj Close"]["SPY"].head())

Date
2007-07-02    115.274590
2007-07-03    115.692307
2007-07-05    115.570786
2007-07-06    116.178352
2007-07-09    116.269455
Name: SPY, dtype: float64


In [3]:
# solve class imbalalnce problem using up-sample minority class

def resample_data(context):
    df_majority = context.indicator.rtn[context.indicator.rtn["SPY_3_pct"]==0]
    df_minority = context.indicator.rtn[context.indicator.rtn["SPY_3_pct"]==1]
    df_minority_upsampled = resample(df_minority,
                                     replace=True,
                                     n_samples=math.floor(len(df_majority)*0.5),
                                     random_state=27
                                     )
    # print(df_minority_upsampled)

    df_upsampled = pd.concat([df_majority, df_minority_upsampled])
    context.indicator.rtn = df_upsampled

In [4]:
def prepare_data(context):
    # fill NA with 10 rolling mean
    for x,y in context.data["Adj Close"].iteritems():
        context.data["Adj Close"][x] = context.data["Adj Close"][x].fillna(context.data["Adj Close"][x].rolling(window = 10, min_periods=1).mean)
    # normalize data
    daily_log_difference(context)

    # SPX dropped more than 5% or 3% in a single day
    context.indicator.rtn["SPY_5_pct"] = [1 if x <=-0.05 else 0 for x in context.indicator.rtn["SPYrtn"]]
    context.indicator.rtn["SPY_3_pct"] = [1 if x <=-0.03 else 0 for x in context.indicator.rtn["SPYrtn"]]

    ## Print to see which day SPY has dropped more than 3%
    # context.indicator.rtn[context.indicator.rtn["SPY_3_pct"]==1] # show the row that is = 1

    # Indicator using 10 day min and max
    for x,y in context.indicator.rtn.iteritems():
        if not ((x == "SPY_5_pct") or (x== "SPY_3_pct")):
            context.indicator.rtn[x + "_10_min"] = context.data["Adj Close"][x[:-3]].shift(-1)/ context.data["Adj Close"][x[:-3]].rolling(10).min() - 1
            context.indicator.rtn[x + "_10_max"] = context.data["Adj Close"][x[:-3]].shift(-1)/ context.data["Adj Close"][x[:-3]].rolling(10).max() - 1

    # make the return shift -1 avoid using the same date data
    for x,y in context.indicator.rtn.iteritems():
        if (x != "SPY_5_pct") or (x!= "SPY_3_pct"):
            context.indicator.rtn[x] = context.indicator.rtn[x].shift(-1)
    #remove nan
    context.indicator.rtn = context.indicator.rtn.dropna(axis=0) # remove any na in the training set
prepare_data(context)

# resample_data(context) # The effect of resample_data is not working too well after testing

c:\users\user\desktop\python project_3.6\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [5]:
target = 'SPY_3_pct' # predict will SPX drop more than 3%
X = context.indicator.rtn.drop(columns=["SPY_3_pct","SPY_5_pct","SPYrtn"])
y = context.indicator.rtn[target]

In [6]:
# random split test and train data into 50%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

# split_ratio = 0.6
# X_train = X[:math.floor(len(X)*split_ratio)]
# X_test = X[math.floor(len(X)*split_ratio):]
# y_train = y[:math.floor(len(y)*split_ratio)]
# y_test = y[math.floor(len(y)*split_ratio):]

In [7]:
print(X_train.isnull().any().sum())
model = DecisionTreeClassifier(max_depth=10)
model.fit(X_train, y_train)

pred_test = model.predict(X_test)
print(pred_test)

print("Accuracy:",accuracy_score(y_test, pred_test, normalize=True, sample_weight=None))
print("Classification Report:", '\n', classification_report(y_test, pred_test))
print("Confusion Matrix:", '\n',confusion_matrix(y_test, pred_test))

treeObj = model.tree_
print (treeObj.node_count)
from sklearn import tree
import matplotlib.pyplot as plt
plt.figure(figsize = (200,300))
tree.plot_tree(model,ax=None, fontsize=100)
plt.show()


0
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

In [8]:
## Optimization byGridSearchCV
# depths = np.arange(3, 10)
# num_leafs = np.arange(10,80,2)
# samples_split= np.arange(2,10)
#
# try_grid = [{'max_depth':depths,
#               'max_leaf_nodes':num_leafs,
#                 "min_samples_split": samples_split
#                     }]
#
# DTM = GridSearchCV(DecisionTreeClassifier(), param_grid = try_grid, cv=10)
# DTM.fit(X,y)

# Cross Validation
print(cross_val_score(DecisionTreeClassifier(), X, y, cv = 20).mean())

0.9472765732416711


In [9]:
# Test model with unseen data
context = Backtest(cash, date_df[0])
context.date_df = date_df
asset_list = "SPY GLD TLT ^VIX ^TNX ^FVX HYG"

context.load_asset_list(asset_list,"2014-01-01",ytd)
prepare_data(context)

target = 'SPY_3_pct'
X = context.indicator.rtn.drop(columns=["SPY_3_pct","SPY_5_pct","SPYrtn"])
y_test = context.indicator.rtn[target]
y_pred = model.predict(X)

print("Accuracy:",accuracy_score(y_test, y_pred, normalize=True, sample_weight=None))
print("Classification Report:", '\n', classification_report(y_test, y_pred))
print("Confusion Matrix:", '\n',confusion_matrix(y_test, y_pred))

### For predicting a financial crisis, having a high recall is more preferable

[*********************100%***********************]  7 of 7 completed
Accuracy: 0.9824451410658307
Classification Report: 
               precision    recall  f1-score   support

         0.0       0.99      0.99      0.99      1569
         1.0       0.47      0.62      0.53        26

    accuracy                           0.98      1595
   macro avg       0.73      0.80      0.76      1595
weighted avg       0.99      0.98      0.98      1595

Confusion Matrix: 
 [[1551   18]
 [  10   16]]


c:\users\user\desktop\python project_3.6\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [10]:
# print(DTM.best_params_)
# print(DTM.best_score_)
# {'max_depth': 5, 'max_leaf_nodes': 10, 'min_samples_split': 9}
# 0.9822950819672132

In [11]:
# from sklearn.linear_model import LogisticRegression
# lr = LogisticRegression(penalty='none', random_state=100, solver='saga', max_iter = 7000)
#
# yact = y.values.ravel()
# yact
# print(cross_val_score(lr, X, yact, cv = 10).mean())

In [12]:
# # import cross validation and other evaluation tool
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import cross_val_predict
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import classification_report
# from sklearn.metrics import confusion_matrix
# from sklearn.model_selection import GridSearchCV
# pred_y = cross_val_predict(lr, X, yact, cv=10)